In [1]:
try:
    import google.colab  # noqa: F401

    %pip install -q dataeval maite-datasets
except Exception:
    pass

In [2]:
import numpy as np
import torch
from maite_datasets.object_detection import VOCDetection
from torchvision.models import ResNet18_Weights, resnet18
from torchvision.transforms.v2 import GaussianNoise

from dataeval import Embeddings, Metadata
from dataeval.core import label_parity
from dataeval.encoders import TorchEmbeddingEncoder
from dataeval.shift import DriftMMD, DriftUnivariate

# Set a random seed
rng = np.random.default_rng(213)

# Set default torch device for notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [3]:
resnet = resnet18(weights=ResNet18_Weights.DEFAULT, progress=False)

# Replace the final fully connected layer with a Linear layer
resnet.fc = torch.nn.Linear(resnet.fc.in_features, 128)

In [4]:
# Load the training dataset
train_ds = VOCDetection("./data", year="2012", image_set="train", download=True)
print(train_ds)
print(f"Image 0 shape: {train_ds[0][0].shape}")

VOCDetection Dataset
--------------------
    Year: 2012
    Transforms: []
    Image Set: train
    Metadata: {'id': 'VOCDetection_train', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'train'}
    Path: /builds/jatic/aria/dataeval/docs/source/notebooks/data/vocdataset/VOCdevkit/VOC2012
    Size: 5717
Image 0 shape: (3, 442, 500)


In [5]:
# Load the "operational" dataset
operational_ds = VOCDetection("./data", year="2012", image_set="val", download=True)
print(operational_ds)
print(f"Image 0 shape: {train_ds[0][0].shape}")

VOCDetection Dataset
--------------------
    Year: 2012
    Transforms: []
    Image Set: val
    Metadata: {'id': 'VOCDetection_val', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'val'}
    Path: /builds/jatic/aria/dataeval/docs/source/notebooks/data/vocdataset/VOCdevkit/VOC2012
    Size: 5823
Image 0 shape: (3, 442, 500)


In [6]:
# Define pretrained model transformations
transforms = ResNet18_Weights.DEFAULT.transforms()

# Create encoder with model and transforms
encoder = TorchEmbeddingEncoder(resnet, batch_size=64, transforms=transforms)

# Create training batches and targets
train_embs = Embeddings(train_ds, encoder=encoder)

# Create operational batches and targets
operational_embs = Embeddings(operational_ds, encoder=encoder)

In [7]:
print(f"({len(train_embs)}, {train_embs[0].shape})")  # (5717, shape)
print(f"({len(operational_embs)}, {operational_embs[0].shape})")  # (5823, shape)

(5717, (128,))
(5823, (128,))


In [8]:
# A type alias for all of the drift detectors
DriftDetector = DriftMMD | DriftUnivariate

# Create a mapping for the detectors to iterate over
detectors: dict[str, DriftDetector] = {
    "MMD": DriftMMD(train_embs),
    "CVM": DriftUnivariate(train_embs, method="cvm"),
    "KS": DriftUnivariate(train_embs, method="ks"),
}

In [9]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(operational_embs).drifted}")

MMD detected drift? False


CVM detected drift? False


KS detected drift? False


In [10]:
# Define transform with added gaussian noise
noisy_transforms = [transforms, GaussianNoise()]

# Create encoder with noisy transforms
noisy_encoder = TorchEmbeddingEncoder(resnet, batch_size=64, transforms=noisy_transforms)

# Applies gaussian noise to images before processing
noisy_embs = Embeddings(operational_ds, encoder=noisy_encoder)

In [11]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(noisy_embs).drifted}")

MMD detected drift? True


CVM detected drift? True


KS detected drift? True


In [12]:
# Get the metadata for each dataset
train_md = Metadata(train_ds)
operational_md = Metadata(operational_ds)

# The VOC dataset has 20 classes
label_parity(train_md.class_labels, operational_md.class_labels, num_classes=20)["p_value"]

0.949856067521638